In [8]:
import cv2, numpy as np, pickle, time
from pathlib import Path

# 1️Cargar clasificador (SVM), codificador y umbral
clf = pickle.load(open("output/recognizer_svm.pickle","rb"))
le  = pickle.load(open("output/label_encoder.pickle","rb"))

THRESH_PATH = Path("output/threshold.pkl")
if THRESH_PATH.exists():
    BEST_T = float(pickle.load(open(THRESH_PATH, "rb")))
    print(f"✅ Umbral cargado desde {THRESH_PATH.name}: {BEST_T:.3f}")
else:
    BEST_T = 0.7  # valor por defecto
    print(f"ℹ️ No se encontró {THRESH_PATH.name}. Usando umbral por defecto: {BEST_T}")

# Cargar embeddings de entrenamiento para control de "desconocido"
EMB_DB_PATH = Path("output/embeddings.pickle")
X_db_norm = None
names_db = None
normalizer = None

if EMB_DB_PATH.exists():
    data_db = pickle.load(open(EMB_DB_PATH, "rb"))
    X_db = np.array(data_db["embeddings"])
    names_db = np.array(data_db["names"])

    # Normalizador del pipeline SVM (el mismo que usaste al entrenar)
    if hasattr(clf, "named_steps") and "normalizer" in clf.named_steps:
        normalizer = clf.named_steps["normalizer"]
        X_db_norm = normalizer.transform(X_db)
        print(f"✅ Embeddings de base cargados: {X_db_norm.shape}")
    else:
        print("⚠️ El clasificador no tiene 'normalizer' en named_steps; no se usará control por distancia.")
else:
    print("ℹ️ No se encontró embeddings.pickle; no se usará control por distancia.")

# Cargar modelo de detección facial (SSD OpenCV)
FACE_PROTO = Path("models/deploy.prototxt")
FACE_MODEL = Path("models/res10_300x300_ssd_iter_140000.caffemodel")
detector = cv2.dnn.readNetFromCaffe(str(FACE_PROTO), str(FACE_MODEL))

def detectar_caras_bgr(img, conf_thresh=0.5):
    """Detecta rostros en una imagen BGR y devuelve [(x1,y1,x2,y2,score), ...]"""
    (h, w) = img.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 1.0, (300, 300),
                                 (104.0, 177.0, 123.0))
    detector.setInput(blob)
    detections = detector.forward()
    boxes = []
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence < conf_thresh:
            continue
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (x1, y1, x2, y2) = box.astype("int")
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(w-1, x2), min(h-1, y2)
        if x2 > x1 and y2 > y1:
            boxes.append((x1, y1, x2, y2, float(confidence)))
    return boxes

# Cargar modelo de embeddings (OpenFace)
EMBED_MODEL = Path("models/openface_nn4.small2.v1.t7")
embedder = cv2.dnn.readNetFromTorch(str(EMBED_MODEL))

def embedding_cara(img, box):
    """Obtiene el embedding (vector 128D) del rostro recortado"""
    (x1, y1, x2, y2, *_) = box
    face = img[y1:y2, x1:x2]
    if face.size == 0:
        return None
    face_blob = cv2.dnn.blobFromImage(cv2.resize(face, (96, 96)),
                                      1.0/255, (96, 96),
                                      (0, 0, 0), swapRB=True, crop=False)
    embedder.setInput(face_blob)
    vec = embedder.forward()
    return vec.flatten()

def _draw_label(img, text, x, y, color):
    """Texto con fondo para mejor visibilidad"""
    (tw, th), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
    cv2.rectangle(img, (x, y - th - 8), (x + tw, y), (0,0,0), -1)
    cv2.putText(img, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

# Reproductor + reconocimiento (video o webcam)
def demo_reproductor(
    source,
    conf_det=0.5,
    conf_cls=None,       # si None, usa BEST_T cargado
    resize_ratio=0.5,    # Factor de rendimiento
    frame_stride=1,      # (de momento sin usar, se podría usar para saltar frames)
    show_fps=True,
    save_out=None
):
    """
    Controles:
      [ESPACIO] Pausa/Reanuda  |  [S] Snapshot  |  [Q]/[ESC] Salir
      [→] Avanza ~1s (solo videos)  |  [←] Retrocede ~1s (solo videos)
    """
    thr = float(conf_cls) if conf_cls is not None else BEST_T
    print(f"🎯 Umbral de reconocimiento (SVM prob): {thr:.2f}")

    # Umbrales adicionales
    thr_gap = 0.15  # gap mínimo entre p1 y p2
    thr_sim = 0.60  # similitud coseno mínima con la base para aceptarlo como conocido

    is_cam = isinstance(source, int)
    cap = cv2.VideoCapture(source)
    if not cap.isOpened():
        raise RuntimeError(f"No se pudo abrir la fuente: {source}")

    fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
    delay_ms = int(1000.0 / fps)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) if not is_cam else -1

    writer = None
    if save_out:
        width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        if resize_ratio:
            width = int(width * resize_ratio)
            height = int(height * resize_ratio)
        fourcc = cv2.VideoWriter_fourcc(*("mp4v" if str(save_out).endswith(".mp4") else "XVID"))
        writer = cv2.VideoWriter(str(save_out), fourcc, fps, (width, height))
        #print(f"💾 Guardando video anotado en: {save_out}")

    snap_dir = Path("snapshots")
    snap_dir.mkdir(parents=True, exist_ok=True)

    paused = False
    prev_time = time.time()
    fps_vis = 0.0

    print("▶ Reproducción iniciada. [ESPACIO]=pausa, [S]=snapshot, [Q]/[ESC]=salir")

    while True:
        if not paused:
            ok, frame = cap.read()
            if not ok:
                print("⏹ Fin del video o error al leer frame.")
                break

            if resize_ratio and 0 < resize_ratio < 1.0:
                frame = cv2.resize(frame, None, fx=resize_ratio, fy=resize_ratio)

            boxes = detectar_caras_bgr(frame, conf_thresh=conf_det)

            for box in boxes:
                (x1, y1, x2, y2, *rest) = box
                vec = embedding_cara(frame, (x1, y1, x2, y2, *rest))
                if vec is None:
                    continue

                # Predicción SVM
                probs = clf.predict_proba([vec])[0]

                order = np.argsort(probs)[::-1]
                j1 = int(order[0])
                p1 = float(probs[j1])

                if len(order) > 1:
                    j2 = int(order[1])
                    p2 = float(probs[j2])
                else:
                    j2, p2 = j1, 0.0

                pred_name = le.classes_[j1]

                # Similitud con embeddings de base (si están disponibles)
                sim_max = None
                name_nn = None
                if (X_db_norm is not None) and (normalizer is not None):
                    # normalizamos el embedding actual igual que los de entrenamiento
                    vec_norm = normalizer.transform([vec])[0]   # (128,)

                    # producto punto porque ya están L2-normalizados → coseno
                    sims = X_db_norm @ vec_norm                 # (N,)
                    idx_max = int(np.argmax(sims))
                    sim_max = float(sims[idx_max])
                    name_nn = names_db[idx_max]

                # Reglas para "Desconocido"
                is_unknown = False

                # Regla de probabilidad y gap
                if (p1 < thr) or ((p1 - p2) < thr_gap):
                    is_unknown = True

                # Regla de similitud en el espacio de embeddings
                if sim_max is not None and sim_max < thr_sim:
                    is_unknown = True

                # (Opcional) si SVM y vecino más cercano discrepan, también lo marcamos sospechoso
                if (name_nn is not None) and (pred_name != name_nn):
                    is_unknown = True

                # Dibujar resultado
                if is_unknown:
                    name = "Desconocido"
                    color = (0, 0, 255)
                else:
                    name = pred_name
                    color = (0, 255, 0)

                texto = f"{name}: {p1:.2f}"
                if sim_max is not None:
                    texto += f" | sim:{sim_max:.2f}"

                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                _draw_label(frame, texto, x1, y1, color)

            if show_fps:
                now = time.time()
                dt = now - prev_time
                if dt > 0:
                    fps_vis = 0.9*fps_vis + 0.1*(1.0/dt) if fps_vis > 0 else 1.0/dt
                prev_time = now
                _draw_label(frame, f"FPS: {fps_vis:.1f}", 10, 25, (0,255,255))

            if writer is not None:
                writer.write(frame)

            cv2.imshow("Reconocimiento facial (Video)", frame)

        key = cv2.waitKey(0 if paused else delay_ms) & 0xFF
        if key in (ord('q'), 27):  # q o ESC
            break
        elif key == ord(' '):      # espacio: pausa/reanuda
            paused = not paused
        elif key == ord('s'):      # snapshot
            snap_path = snap_dir / f"snapshot_{int(time.time())}.jpg"
            cv2.imwrite(str(snap_path), frame)
            print(f"💾 Snapshot guardado: {snap_path}")

    cap.release()
    if writer is not None:
        writer.release()
    cv2.destroyAllWindows()
    print("⏹ Reproducción finalizada.")

✅ Umbral cargado desde threshold.pkl: 0.500
✅ Embeddings de base cargados: (737, 128)


In [9]:

# Main: seleccionar video con Tkinter
import tkinter as tk
from tkinter import filedialog

if __name__ == "__main__":
    # 🧭 Abrir diálogo para seleccionar el video
    root = tk.Tk()
    root.withdraw()  # Ocultar ventana raíz
    video_path = filedialog.askopenfilename(
        title="Selecciona el video a procesar",
        initialdir="video",
        filetypes=[("Archivos de video", "*.mp4 *.avi *.mov *.mkv")]
    )
    if not video_path:
        print("❌ No se seleccionó ningún video. Cancelado.")
        exit()

    src_video = Path(video_path)
    print(f"🎬 Video seleccionado: {src_video.name}")

    # 📁 Nombre de salida (mismo nombre + _anotado)
    save_out =  f"/video/{src_video.stem}_anotado{src_video.suffix}"

    # 🧠 Ejecutar reconocimiento
    demo_reproductor(
        str(src_video),
        conf_det=0.5,
        conf_cls=None,     # o por ejemplo 0.9 si quieres ser más estricto
        resize_ratio=0.8,
        frame_stride=1,
        show_fps=True,
        save_out=str(save_out)
    )

    #print(f"✅ Video anotado guardado en: {save_out}")


🎬 Video seleccionado: Video.mp4
🎯 Umbral de reconocimiento (SVM prob): 0.50
▶ Reproducción iniciada. [ESPACIO]=pausa, [S]=snapshot, [Q]/[ESC]=salir
💾 Snapshot guardado: snapshots\snapshot_1764045859.jpg
💾 Snapshot guardado: snapshots\snapshot_1764045860.jpg
💾 Snapshot guardado: snapshots\snapshot_1764045861.jpg
💾 Snapshot guardado: snapshots\snapshot_1764045863.jpg
💾 Snapshot guardado: snapshots\snapshot_1764045864.jpg
💾 Snapshot guardado: snapshots\snapshot_1764045865.jpg
⏹ Reproducción finalizada.
